In [0]:
import re
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors, doc2vec, FastText
from sklearn.utils import shuffle

from pickle import dump

from nltk.corpus import stopwords

from pymystem3.mystem import Mystem
myst = Mystem()

In [0]:
n = 500 # берём корпуса по n

In [0]:
positive = pd.read_csv('p_tweets.csv', sep=';', header = None, nrows = n, usecols=[0, 3])
# добавляем заголовки из описания базы данных
positive.columns = ["id","ttext"]
print(positive.shape)
print(positive.head())

(500, 2)
                   id                                              ttext
0  408906692374446080  @first_timee хоть я и школота, но поверь, у на...
1  408906692693221377  Да, все-таки он немного похож на него. Но мой ...
2  408906695083954177  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3  408906695356973056  RT @digger2912: "Кто то в углу сидит и погибае...
4  408906761416867842  @irina_dyshkant Вот что значит страшилка :D\nН...


In [0]:
negative = pd.read_csv('n_tweets.csv', sep=';', header = None, nrows = n, usecols=[0, 3])
# добавляем заголовки из описания базы данных
negative.columns = ["id","ttext"]
print(negative.shape)
print(negative.head())

(500, 2)
                   id                                              ttext
0  408906762813579264  на работе был полный пиддес :| и так каждое за...
1  408906818262687744  Коллеги сидят рубятся в Urban terror, а я из-з...
2  408906858515398656  @elina_4post как говорят обещаного три года жд...
3  408906914437685248  Желаю хорошего полёта и удачной посадки,я буду...
4  408906914723295232  Обновил за каким-то лешим surf, теперь не рабо...


In [0]:
# объединяем корпуса
tweets = pd.DataFrame(np.vstack((positive, negative)), columns=["id","ttext"])
tweets = shuffle(tweets)
print(tweets.shape)

(1000, 2)


# Предобработка

In [0]:
# начинаем предобработку
link = re.compile('http\S+') # для удаления ссылок
ment = re.compile('@\S+') # для удаления упоминаний
stop = set(stopwords.words('russian'))
print(stop)

{'нет', 'всю', 'из', 'сейчас', 'ничего', 'у', 'кто', 'какая', 'были', 'между', 'его', 'и', 'такой', 'теперь', 'уже', 'на', 'вы', 'был', 'ты', 'какой', 'при', 'много', 'эту', 'где', 'надо', 'их', 'моя', 'как', 'бы', 'для', 'не', 'ведь', 'мне', 'мы', 'была', 'от', 'к', 'эти', 'чтобы', 'там', 'после', 'тут', 'я', 'да', 'тем', 'один', 'о', 'еще', 'то', 'ли', 'хоть', 'или', 'вдруг', 'разве', 'им', 'так', 'больше', 'над', 'этом', 'иногда', 'впрочем', 'было', 'во', 'под', 'ней', 'сам', 'быть', 'до', 'ей', 'себя', 'нее', 'этот', 'почти', 'чего', 'чем', 'них', 'наконец', 'всего', 'что', 'себе', 'совсем', 'только', 'тогда', 'мой', 'за', 'про', 'нельзя', 'чуть', 'ни', 'когда', 'перед', 'тоже', 'зачем', 'они', 'свою', 'она', 'чтоб', 'через', 'без', 'потому', 'ему', 'конечно', 'того', 'в', 'уж', 'ним', 'он', 'два', 'нас', 'хорошо', 'этой', 'но', 'тебя', 'а', 'вот', 'раз', 'ее', 'нибудь', 'же', 'всех', 'с', 'может', 'меня', 'том', 'со', 'вам', 'лучше', 'будет', 'здесь', 'него', 'ж', 'три', 'если', '

In [0]:
# обрабатываем корпус
for i in range(tweets.shape[0]): # для каждой строки в датафрейме
    print(i)
    text = tweets['ttext'].values[i] # берём текст твита
    #print(text)
    text = re.sub(link, '', text) # удаляем ссылки
    text = re.sub(ment, '', text) # удаляем упоминания
    tokens = []
    for token in re.findall('\w+', text):
        if token.isalpha() and token not in stop: # слово, но не стоп-слово
            #print(token)
            lem = myst.lemmatize(token)[0]
            analiz = myst.analyze(token)[0] # анализируем слово
            #print('analiz', analiz)
            if analiz['analysis']: # если у майстема есть варианты разбора для токена
                #print(len(analiz['analysis'][0]['gr']), analiz['analysis'][0]['gr'])
                gr = re.findall('\w+', analiz['analysis'][0]['gr']) # делим анализ по = и ,
                #print(gr)
                pos = gr[0] # берём часть речи
                #print(lem, pos)
                tokens.append('{}_{}'.format(lem, pos))
            #print()
    #print(tokens, '\n')
    #print()
    tweets['ttext'].values[i] = tokens # перезаписываем значение

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
print(tweets.head())

                     id                                              ttext
540  408909804296351745  [мясорубка_S, пожирать_V, деньги_S, налогоплат...
39   408907082340245504  [каждый_APRO, пират_S, неимоверно_ADV, радоват...
673  408921205669126144  [в_S, горизонт_S, нигде_ADVPRO, чай_S, малина_...
239  408909408694185984  [ну_PART, первый_ANUM, запротоколировать_V, пч...
35   408907042154635264   [электрогенератор_S, иссостойкий_A, побеждать_V]


In [0]:
dump(tweets, open('tweets.pkl', 'wb'))

# Word2Vec

In [0]:
w2v = KeyedVectors.load_word2vec_format('w2v_mystem/ruscorpora_mean_hs.model.bin', binary=True, encoding='utf-8')

In [0]:
dim = w2v.vector_size

In [0]:
w2v.vocab

{'это_SPRO': <gensim.models.keyedvectors.Vocab at 0x1a42de12f60>,
 'весь_APRO': <gensim.models.keyedvectors.Vocab at 0x1a42de12be0>,
 'человек_S': <gensim.models.keyedvectors.Vocab at 0x1a4205dd160>,
 'мочь_V': <gensim.models.keyedvectors.Vocab at 0x1a42136e128>,
 'год_S': <gensim.models.keyedvectors.Vocab at 0x1a42aa75da0>,
 'сказать_V': <gensim.models.keyedvectors.Vocab at 0x1a42de8a0f0>,
 'время_S': <gensim.models.keyedvectors.Vocab at 0x1a42de8a080>,
 'говорить_V': <gensim.models.keyedvectors.Vocab at 0x1a42de8a240>,
 'становиться_V': <gensim.models.keyedvectors.Vocab at 0x1a42de8a208>,
 'знать_V': <gensim.models.keyedvectors.Vocab at 0x1a42de8a128>,
 'самый_APRO': <gensim.models.keyedvectors.Vocab at 0x1a42de8a1d0>,
 'жизнь_S': <gensim.models.keyedvectors.Vocab at 0x1a42de8a160>,
 'дело_S': <gensim.models.keyedvectors.Vocab at 0x1a42de8a198>,
 'день_S': <gensim.models.keyedvectors.Vocab at 0x1a42de8a0b8>,
 'рука_S': <gensim.models.keyedvectors.Vocab at 0x1a42de8a278>,
 'первый_ANU

In [0]:
vecs = pd.DataFrame(index = [i for i in range(tweets.shape[0])], columns = [d for d in range(dim)])
print(vecs.head(2))
print(vecs.shape)

   0    1    2    3    4    5    6    7    8    9   ...   290  291  292  293  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   

   294  295  296  297  298  299  
0  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 300 columns]
(1000, 300)


In [0]:
# функция получения вектора текста
def vectorize (tokens): # на вход -- список токенов
    s_vec = np.zeros(dim)
    #print(s_vec)
    #print(s_vec.shape)
    for token in tokens:
        #print(token)
        if token in w2v.vocab: # если слово есть в словаре word2vec
            w_vec = w2v[token] # получаем его векто
            #print(type(w_vec), w_vec)
            s_vec = np.vstack((s_vec, w_vec)) # добавляем в массив к остальным токенам
            #print(s_vec)
            #print(s_vec.shape)
            #print()
    s_vec = s_vec[1:] # отбрасываем нулевую строку
    s_vec = s_vec.max(axis=0) # собираем массив в один вектор, выбирая максимальное значение
    return s_vec

In [0]:
for t in range(tweets.shape[0]): # для каждой строки в датафрейме
    text = tweets['ttext'].values[i] # берём текст твита
    s_vec = vectorize(text) # векторизуем
    #print(s_vec)
    #print(s_vec.shape)
    vecs.values[t] = s_vec # заменяем строчки на наш вектор

In [0]:
print(vecs.head(2))

         0          1          2         3          4          5          6    \
0  0.0849773  0.0794015  0.0534515  0.104456  0.0294858  0.0727509  0.0506909   
1  0.0849773  0.0794015  0.0534515  0.104456  0.0294858  0.0727509  0.0506909   

        7          8         9      ...           290        291       292  \
0  0.111664  0.0548081  0.116661    ...      0.137362  0.0952921  0.176735   
1  0.111664  0.0548081  0.116661    ...      0.137362  0.0952921  0.176735   

         293        294        295        296        297        298        299  
0  0.0658063  0.0147958  0.0449803  0.0640902  0.0829611  0.0589186  0.0863296  
1  0.0658063  0.0147958  0.0449803  0.0640902  0.0829611  0.0589186  0.0863296  

[2 rows x 300 columns]


In [0]:
w2v_tweets = pd.concat([tweets, vecs], axis=1) # объединяем текст и векторы
print(w2v_tweets.head(2))
print(w2v_tweets.shape)

                   id                                              ttext  \
0  408906692374446080  [школоть_V, поверять_V, самый_APRO, общество_S...   
1  408906692693221377  [да_PART, таки_PART, немного_ADV, похожий_A, н...   

           0          1          2         3          4          5          6  \
0  0.0849773  0.0794015  0.0534515  0.104456  0.0294858  0.0727509  0.0506909   
1  0.0849773  0.0794015  0.0534515  0.104456  0.0294858  0.0727509  0.0506909   

          7    ...           290        291       292        293        294  \
0  0.111664    ...      0.137362  0.0952921  0.176735  0.0658063  0.0147958   
1  0.111664    ...      0.137362  0.0952921  0.176735  0.0658063  0.0147958   

         295        296        297        298        299  
0  0.0449803  0.0640902  0.0829611  0.0589186  0.0863296  
1  0.0449803  0.0640902  0.0829611  0.0589186  0.0863296  

[2 rows x 302 columns]
(1000, 302)


In [0]:
dump(w2v_tweets, open('w2v_tweets.pkl', 'wb'))

# Doc2Vec

In [0]:
tags = [i for i in range(tweets.shape[0])] # список документов

doc_tag = [] # список списков токенов, соотнесённых с документами
for tag in tags:
        doc_tag.append(doc2vec.TaggedDocument(tweets['ttext'].values[i], [tags[tag]]))

In [0]:
print(doc_tag[:10])

[TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[0]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[1]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[2]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[3]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[4]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[5]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убеждаться_V', 'ошибаться_V', 'выбор_S', 'кумир_S', 'деми_S', 'спасибо_S'], tags=[6]), TaggedDocument(words=['каждый_APRO', 'день_S', 'убежда

In [0]:
dim = 50

In [0]:
d2v = doc2vec.Doc2Vec(size=dim, min_count=2, iter=55) 
d2v.build_vocab(doc_tag) # строим словарь
d2v.train(doc_tag, total_examples=d2v.corpus_count, epochs=d2v.iter)

c:\program files\python36\lib\site-packages\gensim\models\doc2vec.py:531: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
c:\program files\python36\lib\site-packages\gensim\models\doc2vec.py:535: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
vecs = pd.DataFrame(index = [i for i in range(tweets.shape[0])], columns = [d for d in range(dim)])
print(vecs.head(2))

    0    1    2    3    4    5    6    7    8    9  ...    40   41   42   43  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   

    44   45   46   47   48   49  
0  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 50 columns]


In [0]:
for t in range(tweets.shape[0]): # для каждой строки в датафрейме
    s_vec = d2v.infer_vector(doc_tag[t].words) # векторизуем текст
    #print(s_vec)
    #print(s_vec.shape)
    vecs.values[t] = s_vec # заменяем строчки на наш вектор

In [0]:
print(vecs.head(2))

          0          1         2          3          4          5          6   \
0 -0.0637962  0.0926807  -0.04535 -0.0879545 -0.0152108  0.0210416  0.0552975   
1 -0.0630521  0.0961158 -0.046553 -0.0937671 -0.0164633  0.0298075  0.0539274   

           7          8          9     ...             40         41  \
0 -0.00574748  0.0395987 -0.0568197    ...      0.0165672   0.036554   
1 -0.00530662  0.0330972 -0.0531901    ...      0.0217488  0.0353151   

          42         43         44        45         46         47         48  \
0  0.0812612 -0.0353729  0.0837089  0.105266  0.0415174  0.0199152  0.0384459   
1  0.0649798 -0.0474468  0.0777285  0.111102  0.0476141  0.0152661  0.0288602   

          49  
0 -0.0133227  
1 -0.0142199  

[2 rows x 50 columns]


In [0]:
d2v_tweets = pd.concat([tweets, vecs], axis=1) # объединяем текст и векторы
print(d2v_tweets.head(2))
print(d2v_tweets.shape)

                   id                                              ttext  \
0  408906692374446080  [школоть_V, поверять_V, самый_APRO, общество_S...   
1  408906692693221377  [да_PART, таки_PART, немного_ADV, похожий_A, н...   

           0          1         2          3          4          5          6  \
0 -0.0637962  0.0926807  -0.04535 -0.0879545 -0.0152108  0.0210416  0.0552975   
1 -0.0630521  0.0961158 -0.046553 -0.0937671 -0.0164633  0.0298075  0.0539274   

            7    ...             40         41         42         43  \
0 -0.00574748    ...      0.0165672   0.036554  0.0812612 -0.0353729   
1 -0.00530662    ...      0.0217488  0.0353151  0.0649798 -0.0474468   

          44        45         46         47         48         49  
0  0.0837089  0.105266  0.0415174  0.0199152  0.0384459 -0.0133227  
1  0.0777285  0.111102  0.0476141  0.0152661  0.0288602 -0.0142199  

[2 rows x 52 columns]
(1000, 52)


In [0]:
dump(d2v_tweets, open('d2v_tweets.pkl', 'wb'))

# Fasttext

In [0]:
dim = 50

In [0]:
ftx = FastText(tweets['ttext'].values, size=dim, window=3, min_count=1, iter=10)

In [0]:
ftx.wv.vocab

{'мясорубка_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d470>,
 'пожирать_V': <gensim.models.keyedvectors.Vocab at 0x1a44972dd68>,
 'деньги_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d3c8>,
 'налогоплательщик_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d400>,
 'каждый_APRO': <gensim.models.keyedvectors.Vocab at 0x1a44972da90>,
 'пират_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d208>,
 'неимоверно_ADV': <gensim.models.keyedvectors.Vocab at 0x1a44972de80>,
 'радовать_V': <gensim.models.keyedvectors.Vocab at 0x1a44972df98>,
 'свой_APRO': <gensim.models.keyedvectors.Vocab at 0x1a44972da58>,
 'политический_A': <gensim.models.keyedvectors.Vocab at 0x1a44972d7f0>,
 'позиция_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d128>,
 'отсутствие_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d4e0>,
 'понимание_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d2e8>,
 'факт_S': <gensim.models.keyedvectors.Vocab at 0x1a44972d4a8>,
 'в_S': <gensim.models.keyedvectors.Vo

In [0]:
ftx['школоть_V']

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.9663051 , -1.1686101 ,  0.7733092 ,  0.49221018, -1.057372  ,
       -0.7787898 ,  0.64398235, -0.53360933, -0.36425313,  0.6796667 ,
       -0.11008651,  0.5829438 , -1.0026844 , -1.3798552 ,  0.9177173 ,
        0.68687576, -0.92937976,  0.16431542,  0.12238119, -0.93380904,
       -0.10401317,  0.69880563,  0.00944513,  0.6997781 , -0.39095646,
       -0.8788594 , -0.8580042 ,  0.62218755, -0.12286377,  0.84888846,
        0.83932143,  1.8911275 , -0.8442667 ,  0.78296   , -0.44310552,
       -0.14985654,  0.53061867, -0.71564066,  0.7351906 ,  0.17501654,
       -1.0616587 , -1.5243045 ,  0.3860594 , -0.00690492,  0.51177156,
       -0.21251373,  0.81045145, -0.7686418 ,  0.9814982 ,  0.13667418],
      dtype=float32)

In [0]:
vecs = pd.DataFrame(index = [i for i in range(tweets.shape[0])], columns = [d for d in range(dim)])
print(vecs.head(2))

    0    1    2    3    4    5    6    7    8    9  ...    40   41   42   43  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN  NaN  NaN   

    44   45   46   47   48   49  
0  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 50 columns]


In [0]:
# функция получения вектора текста
def vectorize (tokens): # на вход -- список токенов
    s_vec = np.zeros(dim)
    #print(s_vec)
    #print(s_vec.shape)
    for token in tokens:
        #print(token)
        if token in ftx.wv.vocab: # если слово есть в словаре fasttext
            w_vec = ftx[token] # получаем его вектор
            #print(type(w_vec), w_vec)
            s_vec = np.vstack((s_vec, w_vec)) # добавляем в массив к остальным токенам
            #print(s_vec)
            #print(s_vec.shape)
            #print()
    s_vec = s_vec[1:] # отбрасываем нулевую строку
    s_vec = s_vec.max(axis=0) # собираем массив в один вектор, выбирая максимальное значение
    return s_vec

In [0]:
for t in range(tweets.shape[0]): # для каждой строки в датафрейме
    text = tweets['ttext'].values[i] # берём текст твита
    s_vec = vectorize(text) # векторизуем
    #print(s_vec)
    #print(s_vec.shape)
    vecs.values[t] = s_vec # заменяем строчки на наш вектор

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [0]:
print(vecs.head(2))

        0         1         2         3         4         5         6   \
0  1.06129 -0.761827  0.853798  0.542376 -0.688801 -0.500383  0.704201   
1  1.06129 -0.761827  0.853798  0.542376 -0.688801 -0.500383  0.704201   

         7         8         9     ...           40        41        42  \
0 -0.348896 -0.239841  0.745683    ...    -0.690921 -0.989647  0.426885   
1 -0.348896 -0.239841  0.745683    ...    -0.690921 -0.989647  0.426885   

           43        44        45        46        47       48        49  
0 -0.00202123  0.564508 -0.145085  0.888179 -0.503026  1.07535  0.154853  
1 -0.00202123  0.564508 -0.145085  0.888179 -0.503026  1.07535  0.154853  

[2 rows x 50 columns]


In [0]:
ftx_tweets = pd.concat([tweets, vecs], axis=1) # объединяем текст и векторы
print(ftx_tweets.head(2))
print(ftx_tweets.shape)

                   id                                              ttext  \
0  408906692374446080  [школоть_V, поверять_V, самый_APRO, общество_S...   
1  408906692693221377  [да_PART, таки_PART, немного_ADV, похожий_A, н...   

         0         1         2         3         4         5         6  \
0  1.06129 -0.761827  0.853798  0.542376 -0.688801 -0.500383  0.704201   
1  1.06129 -0.761827  0.853798  0.542376 -0.688801 -0.500383  0.704201   

          7    ...           40        41        42          43        44  \
0 -0.348896    ...    -0.690921 -0.989647  0.426885 -0.00202123  0.564508   
1 -0.348896    ...    -0.690921 -0.989647  0.426885 -0.00202123  0.564508   

         45        46        47       48        49  
0 -0.145085  0.888179 -0.503026  1.07535  0.154853  
1 -0.145085  0.888179 -0.503026  1.07535  0.154853  

[2 rows x 52 columns]
(1000, 52)


In [0]:
dump(ftx_tweets, open('ftx_tweets.pkl', 'wb'))